In [ ]:
import paths
from utils import *

from models.DeepLabTL import *
from models.DeepLabResNet50 import *
from models.DeepLabResNet101 import *
from models.DeepLabMobileNet import *
from models.ResNet import *


import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import cv2
import numpy as np

from configparser import ConfigParser
import configparser
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, f1_score, roc_auc_score
from collections import defaultdict
from pathlib import Path
from typing import Any, Callable, Optional
from PIL import Image
import shutil
import copy
import csv
import time
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
import torchvision.transforms as T
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.datasets.vision import VisionDataset
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision import models
import torch.nn.functional as F

In [ ]:
train_df = pd.read_csv(paths.train_path)
meta_df = pd.read_csv(paths.meta_path)
test_df = pd.read_csv(paths.test_path)

In [ ]:
new_train = extract_20_classes(paths.train_folder)
new_train = '/content/Train/new_dataset/Train'

In [ ]:
# creating new csv files
meta_csv = filter_and_create_csv(paths.meta_path, 'Meta.csv')
test_csv = filter_and_create_csv(paths.test_path, 'Test.csv')
train_csv = filter_and_create_csv(paths.train_path, 'Train.csv')

In [ ]:
# reading csv files using pandas library
read_meta_csv = pd.read_csv(meta_csv)
read_test_csv = pd.read_csv(test_csv)
read_train_csv = pd.read_csv(train_csv)

In [ ]:
# Acessing classId column inside csv files
class_counts_Test = read_test_csv['ClassId'].value_counts()
class_counts_Train = read_train_csv['ClassId'].value_counts()
class_counts_Meta = read_meta_csv['ClassId'].value_counts()

class_counts_Meta = class_counts_Meta.sort_index()
class_counts_Train = class_counts_Train.sort_index()
class_counts_Test = class_counts_Test.sort_index()

# Create a figure with three subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Plotting for 'Meta' dataset
axs[0].bar(class_counts_Meta.index.astype(int),
           class_counts_Meta.values, color='blue')
axs[0].set_title('Meta Dataset')
axs[0].set_xlabel('Class Labels')
axs[0].set_ylabel('Number of Samples')

# Plotting for 'Train' dataset
axs[1].bar(class_counts_Train.index.astype(int), class_counts_Train.values)
axs[1].set_title('Train Dataset')
axs[1].set_xlabel('Class Labels')
axs[1].set_ylabel('Number of Samples')

# Plotting for 'Test' dataset
axs[2].bar(class_counts_Test.index.astype(int), class_counts_Test.values)
axs[2].set_title('Test Dataset')
axs[2].set_xlabel('Class Labels')
axs[2].set_ylabel('Number of Samples')

axs[0].bar(class_counts_Meta.index.astype(int), class_counts_Meta.values)
axs[0].set_title('Meta Dataset')
axs[0].set_xlabel('Class Labels')
axs[0].set_ylabel('Number of Samples')

custom_ticks = [i for i in range(0, len(class_counts_Meta.index), 1)]
custom_labels = [str(i) for i in custom_ticks]
plt.tight_layout()
plt.show()